# Notebook Details

In this notebook we will give short term memory.

We have 2 tools:
- 1. Calculator Tools.
- 2. wikipwdia web search.

We will store old conversion to a list and pass this histoy as context of the query

## Defining Tools

In [1]:
# import Necessary Library
import requests
import ollama

In [2]:
# 1. Simple calculator tool
def calculator_tool(query):
    """A simple calculator tool."""
    try:
        # Keep only numbers, operators, parentheses, and dots
        cleaned_expr = query.split('=')[0]  # Remove anything after '='
        cleaned_expr = "".join(cleaned_expr).strip()
        return str(eval(cleaned_expr))
    except Exception as e:
        return f"Error in calculation: {e}"

In [3]:
# 2. Wikipedia search tool
def web_search(query):
    url = "https://en.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "list": "search",
        "srsearch": query,
        "format": "json"
    }
    try:
        response = requests.get(url, params=params)
        data = response.json()
        if "query" in data and "search" in data["query"]:
            results = [item['snippet'] for item in data['query']['search']]
            return "\n".join(results) if results else "No results found on Wikipedia."
        else:
            return "No results found."
    except Exception as e:
        return f"Error searching Wikipedia: {e}"

## Defining Memory

In [4]:
memory = [
    {
        'role': 'system',
        'content': """
                    You are a helpful AI assistant. You can either perform calculations or search the web.
                    - CALCULATE: <math expression>
                    - SEARCH: <search query>
                    - ANSWER: <your answer>

                    Always use the information from previous conversation if relevant.
                    Output ONLY one of the above actions on the FIRST line.
                    Do not explain your reasoning. Do not add extra commentary.

                    Output ONLY one of the above actions on the FIRST line. 
                    Do not explain your reasoning. Do not add extra commentary.
                    
                    Example:
                    Q: What is 12*(5+2)?
                    A: CALCULATE: 12*(5+2)
                    
                    Q: Who is the president of the USA?
                    A: SEARCH: Who is the president of the USA?
                    
                    Q: Tell me a joke about Cats.
                    A: ANSWER: Sure, here is a joke...
                    """
    }
]

## Defining Agent

In [9]:
def agent(query, model="llama3.2:latest"):
    # Adding user query to memory
    memory.append({"role": "user", "content": query})

    # Ask LLM with conversation history
    decision_response = ollama.chat(
        model=model,
        messages=memory
    )

    # Extracting the tools name from decision response
    decision = decision_response['message']['content'].strip()

    # save model's reasoning memory
    memory.append({"role": "assistant", "content": decision})

    # Get  first line of (action)
    first_line = decision.split('\n')[0].strip()

    print(f"\tDecision made by LLM: {first_line}")

    # Exectuing the tools
    if "CALCULATE" in first_line:
        expression = first_line.split("CALCULATE:", 1)[1].strip()
        result = calculator_tool(expression)
    elif "SEARCH:" in first_line:
        search_query = first_line.split("SEARCH:")[1].strip()
        result = web_search(search_query)
    elif "ANSWER:" in first_line:
        result = first_line.split("ANSWER:")[1].strip()
    else:
        result = "Invalid action specified."
    
    # Savinig Result to memory
    memory.append({"role": "assistant", "content": result})

    return result


In [10]:

print(agent("Who is the ceo of Google?"))

	Decision made by LLM: A: CALCULATE: "Sundar Pichai"
Sundar Pichai


In [11]:
print(memory)

[{'role': 'system', 'content': '\n                    You are a helpful AI assistant. You can either perform calculations or search the web.\n                    - CALCULATE: <math expression>\n                    - SEARCH: <search query>\n                    - ANSWER: <your answer>\n\n                    Always use the information from previous conversation if relevant.\n                    Output ONLY one of the above actions on the FIRST line.\n                    Do not explain your reasoning. Do not add extra commentary.\n\n                    Output ONLY one of the above actions on the FIRST line. \n                    Do not explain your reasoning. Do not add extra commentary.\n\n                    Example:\n                    Q: What is 12*(5+2)?\n                    A: CALCULATE: 12*(5+2)\n\n                    Q: Who is the president of the USA?\n                    A: SEARCH: Who is the president of the USA?\n\n                    Q: Tell me a joke about Cats.\n           

In [12]:
print(agent("How old is he?"))

	Decision made by LLM: A: ANSWER: Sundar Pichai was born on December 12, 1972. He is currently 51 years old.
Sundar Pichai was born on December 12, 1972. He is currently 51 years old.
